# 빅데이터 분석 Mini Project
### 멜론 2010년~2019년 월간 차트 크롤링

* Python 3 기반
* selenium의 webdriver, Beautifulsoup, requests 모듈 사용
* 전체적인 틀을 제외하고 대부분 직접 코딩함

In [5]:
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import requests
import re

In [8]:
header = {'User-Agent': ''}
d = webdriver.Chrome('./Downloads/chromedriver')
d.implicitly_wait(3)
d.get('http://www.melon.com/chart/index.htm')
d.get("http://www.melon.com/chart/search/index.htm")
d.find_element_by_xpath('//*[@id="d_chart_search"]/div/h4[1]/a').click() #멜론의 차트파인더에서 주간차트 선택

req = requests.get('http://www.melon.com/chart/search/index.htm')
html = req.text
soup = BeautifulSoup(html, "html.parser")

#2010년대만 선택 3번째 div sector의 인덱스가 연대-연도-월-주간-차트 순을 뜻함            
age_xpath = '//*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[1]/span/label' 
age = d.find_element_by_xpath(age_xpath)
age.click()

f=open('melon_weekly.txt','w',-1,'UTF-8')

for i in range(1, 11): #연도 10번 선택
    
    try:
            year_xpath = '//*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[' + str(i) + ']/span/label' #년도 선택
            year = d.find_element_by_xpath(year_xpath)
            year.click()
            print(year.text)

    except:
            print("year_xpath not found")
            pass
    
    sleep(1)
    for i in range(1, 13): #월 12번 선택


        try:
            month_xpath = '//*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li[' + str(i) + ']/span/label' #월선택
            month = d.find_element_by_xpath(month_xpath)
            month.click()
            print(month.text)

        except:
            print("month_xpath not found")
            pass
        
        sleep(1)
        for i in range(1, 6): #주간 5번 선택
            
            try:
                week_xpath = '//*[@id="d_chart_search"]/div/div/div[4]/div[1]/ul/li[' + str(i) + ']/span/label' #주선택
                week = d.find_element_by_xpath(week_xpath)
                week.click()
                print(week.text)

            except:
                print("week_xpath not found")
                break
            pass
            sleep(2)
            
            classCd = d.find_element_by_xpath('//label[@for = "gnr_1"]')
            if '가요' not in classCd.text:
                classCd = d.find_element_by_xpath('//label[@for = "gnr_2"]')

            classCd.click()
            sleep(1)
            
            d.find_element_by_xpath('//*[@id="d_srch_form"]/div[2]/button/span/span').click() #검색 버튼 누르기
            sleep(3)
            
            artists = d.find_elements_by_xpath('//*[@id="lst50"]/td[4]/div/div/div[2]/div[1]/a[1]')
            
            changed_ranks = d.find_elements_by_xpath('//*[@id="lst50"]/td[2]/div/span[3]/span[1]') #어떻게 변동되었는지 (상승,하락,유지,NEW)
            
            #변동된 순위, NEW(차트진입)인 경우 변동된 순위가 없어서 changed_ranks_num과 changed_ranks의 인덱스가 서로 mapping되지 않기 때문에 밑에는 보완하는 코드
            changed_ranks_num = list()

            a=0
            for i,c in zip(range(0,50),changed_ranks):
                temp = d.find_elements_by_xpath('//*[@id="lst50"]/td[2]/div/span[3]/span[2]') 
                if 'NEW' in c.text:
                    changed_ranks_num.append(str(0))
                    a=a+1 #NEW가 몇개있나
                else : changed_ranks_num.append(temp[i-a].text)

            for i,artist,changed_rank, changed_rank_num in zip(range(1,51), artists, changed_ranks, changed_ranks_num):#1위부터 50위까지에 대해서 수행
                
                t = "" #곡명 저장하는 변수

                try:
                    t = d.find_element_by_xpath('//*[@id="lst50"][' + str(i) + ']/td[4]/div/div/div[1]/span/strong/a').text #곡명 추가
                except: 
                    t = d.find_element_by_xpath('//*[@id="lst50"][' + str(i) + ']/td[4]/div/div/div[1]/span/strong/span').text #재생불가능한 곡명(태그가 다름)
                
                if '19금' in t:
                    t = t.replace('19금', '')

                print("차트 연도:", year.text)
                print("달:", month.text)
                print("주:", week.text)
                print("순위:", i)
                print("제목:", t)
                print("아티스트:", artist.text)
                print(changed_rank_num + ' ' + changed_rank.text)
                print("----")
                
                data = "{0}//{1}//{2}//{3:3d}//{4}//{5}//{6}//{7}".format(re.sub('[^0-9]', '', year.text),month.text,week.text,i,t,artist.text,changed_rank_num,changed_rank.text)
                f.write(data + '\n')
                

            d.find_element_by_xpath('//*[@id="frm"]/div[2]/span/a').click() #51-100위권 페이지 진입
            sleep(3) 
            
            artists = d.find_elements_by_xpath('//*[@id="lst100"]/td[4]/div/div/div[2]/div[1]/a[1]') #가수 리스트로
            
            changed_ranks = d.find_elements_by_xpath('//*[@id="lst100"]/td[2]/div/span[3]/span[1]') #어떻게 변동되었는지
            
            changed_ranks_num = list()

            a=0
            for i,c in zip(range(0,50),changed_ranks):
                temp = d.find_elements_by_xpath('//*[@id="lst100"]/td[2]/div/span[3]/span[2]') #변동된 순위 임시배열(New인경우 변동된 순위가 존재 x 따라서 배열의 크기를 changed_ranks와 맞춰줘야함)
                if 'NEW' in c.text:
                    changed_ranks_num.append(str(0))
                    a=a+1 #NEW가 몇개있나
                else : changed_ranks_num.append(temp[i-a].text)
     
            for i,artist,changed_rank, changed_rank_num in zip(range(1,51), artists, changed_ranks, changed_ranks_num):#51위부터 100위까지에 대해서 수행
                
                t = "" #곡명 저장하는 변수

                try:
                    t = d.find_element_by_xpath('//*[@id="lst100"][' + str(i) + ']/td[4]/div/div/div[1]/span/strong/a').text #곡명 추가
                except: 
                    t = d.find_element_by_xpath('//*[@id="lst100"][' + str(i) + ']/td[4]/div/div/div[1]/span/strong/span').text #재생불가능한 곡명(태그가 다름)
                
                if '19금' in t:
                    t = t.replace('19금', '')

                print("차트 연도:", year.text)
                print("달:", month.text)
                print("주:", week.text)
                print("순위:", i+50)
                print("제목:", t)
                print("아티스트:", artist.text)
                print(changed_rank_num + ' ' + changed_rank.text)
                print("----")
                
                data = "{0}//{1}//{2}//{3:3d}//{4}//{5}//{6}//{7}".format(re.sub('[^0-9]', '', year.text),month.text,week.text,i+50,t,artist.text,changed_rank_num,changed_rank.text)
                f.write(data + '\n')
               
        
f.close()

2012년
08월
07.29~08.04
차트 연도: 2012년
달: 08월
주: 07.29~08.04
순위: 1
제목: 강남스타일
아티스트: 싸이 (PSY)
0 순위 동일
----
차트 연도: 2012년
달: 08월
주: 07.29~08.04
순위: 2
제목: Loving U (러빙유)
아티스트: 씨스타
1 단계 상승
----
차트 연도: 2012년
달: 08월
주: 07.29~08.04
순위: 3
제목: I Love You
아티스트: 2NE1
1 단계 하락
----
차트 연도: 2012년
달: 08월
주: 07.29~08.04
순위: 4
제목: Only One
아티스트: 보아 (BoA)
0 순위 동일
----
차트 연도: 2012년
달: 08월
주: 07.29~08.04
순위: 5
제목: 남자도 우나요
아티스트: 다비치
0 NEW
----
차트 연도: 2012년
달: 08월
주: 07.29~08.04
순위: 6
제목: DAY BY DAY
아티스트: 티아라
1 단계 하락
----
차트 연도: 2012년
달: 08월
주: 07.29~08.04
순위: 7
제목: Midnight (별 헤는 밤)
아티스트: 비스트
1 단계 하락
----
차트 연도: 2012년
달: 08월
주: 07.29~08.04
순위: 8
제목: 뜨거운 안녕 (Feat. 성시경)
아티스트: 싸이 (PSY)
0 순위 동일
----
차트 연도: 2012년
달: 08월
주: 07.29~08.04
순위: 9
제목: illa illa (일라 일라)
아티스트: JUNIEL
0 순위 동일
----
차트 연도: 2012년
달: 08월
주: 07.29~08.04
순위: 10
제목: 정말로 사랑한다면
아티스트: 버스커 버스커
3 단계 하락
----
차트 연도: 2012년
달: 08월
주: 07.29~08.04
순위: 11
제목: 아름다운 밤이야
아티스트: 비스트
1 단계 상승
----
차트 연도: 2012년
달: 08월
주: 07.29~08.04
순위: 12
제목: 한 여름 밤의 꿈
아티스트: 브라운아이드걸스
1 

29 단계 하락
----
차트 연도: 2012년
달: 08월
주: 07.29~08.04
순위: 98
제목: 니가 쉬는 날
아티스트: 비스트
23 단계 하락
----
차트 연도: 2012년
달: 08월
주: 07.29~08.04
순위: 99
제목: 기다려주세요
아티스트: 버스커 버스커
11 단계 하락
----
차트 연도: 2012년
달: 08월
주: 07.29~08.04
순위: 100
제목: Dream Girl
아티스트: 비스트
23 단계 하락
----
08.05~08.11
차트 연도: 2012년
달: 08월
주: 08.05~08.11
순위: 1
제목: 강남스타일
아티스트: 싸이 (PSY)
0 순위 동일
----
차트 연도: 2012년
달: 08월
주: 08.05~08.11
순위: 2
제목: Loving U (러빙유)
아티스트: 씨스타
0 순위 동일
----
차트 연도: 2012년
달: 08월
주: 08.05~08.11
순위: 3
제목: I Love You
아티스트: 2NE1
0 순위 동일
----
차트 연도: 2012년
달: 08월
주: 08.05~08.11
순위: 4
제목: Only One
아티스트: 보아 (BoA)
0 순위 동일
----
차트 연도: 2012년
달: 08월
주: 08.05~08.11
순위: 5
제목: 남자도 우나요
아티스트: 다비치
0 순위 동일
----
차트 연도: 2012년
달: 08월
주: 08.05~08.11
순위: 6
제목: 뜨거운 안녕 (Feat. 성시경)
아티스트: 싸이 (PSY)
2 단계 상승
----
차트 연도: 2012년
달: 08월
주: 08.05~08.11
순위: 7
제목: Midnight (별 헤는 밤)
아티스트: 비스트
0 순위 동일
----
차트 연도: 2012년
달: 08월
주: 08.05~08.11
순위: 8
제목: 아름다운 밤이야
아티스트: 비스트
3 단계 상승
----
차트 연도: 2012년
달: 08월
주: 08.05~08.11
순위: 9
제목: DAY BY DAY
아티스트: 티아라
3 단계 하락
----

달: 08월
주: 08.05~08.11
순위: 95
제목: 너뿐이야 (You`re The One)
아티스트: 박진영
10 단계 하락
----
차트 연도: 2012년
달: 08월
주: 08.05~08.11
순위: 96
제목: 우린 좀 달라
아티스트: 이블
0 NEW
----
차트 연도: 2012년
달: 08월
주: 08.05~08.11
순위: 97
제목: The Shadow
아티스트: 보아 (BoA)
9 단계 하락
----
차트 연도: 2012년
달: 08월
주: 08.05~08.11
순위: 98
제목: 오늘같은 밤이면
아티스트: 2BIC(투빅)
9 단계 하락
----
차트 연도: 2012년
달: 08월
주: 08.05~08.11
순위: 99
제목: 좋아보여 (Feat. 검정치마)
아티스트: 버벌진트
0 NEW
----
차트 연도: 2012년
달: 08월
주: 08.05~08.11
순위: 100
제목: EGO
아티스트: BIGBANG
10 단계 하락
----
08.13~08.19
차트 연도: 2012년
달: 08월
주: 08.13~08.19
순위: 1
제목: 강남스타일
아티스트: 싸이 (PSY)
0 순위 동일
----
차트 연도: 2012년
달: 08월
주: 08.13~08.19
순위: 2
제목: Loving U (러빙유)
아티스트: 씨스타
0 순위 동일
----
차트 연도: 2012년
달: 08월
주: 08.13~08.19
순위: 3
제목: Only One
아티스트: 보아 (BoA)
1 단계 상승
----
차트 연도: 2012년
달: 08월
주: 08.13~08.19
순위: 4
제목: I Love You
아티스트: 2NE1
1 단계 하락
----
차트 연도: 2012년
달: 08월
주: 08.13~08.19
순위: 5
제목: 아름다운 밤이야
아티스트: 비스트
3 단계 상승
----
차트 연도: 2012년
달: 08월
주: 08.13~08.19
순위: 6
제목: 남자도 우나요
아티스트: 다비치
1 단계 하락
----
차트 연도: 2012년
달: 08월
주: 08

차트 연도: 2012년
달: 08월
주: 08.13~08.19
순위: 92
제목: FANTASTIC BABY
아티스트: BIGBANG
11 단계 하락
----
차트 연도: 2012년
달: 08월
주: 08.13~08.19
순위: 93
제목: 킹콩샤워 (Feat. G.NA)
아티스트: Fresh Boyz
0 NEW
----
차트 연도: 2012년
달: 08월
주: 08.13~08.19
순위: 94
제목: 붉은노을 (Feat. 김지수 & Egobomb)
아티스트: MC 스나이퍼
0 NEW
----
차트 연도: 2012년
달: 08월
주: 08.13~08.19
순위: 95
제목: 폭풍속으로 (She`s Gone)
아티스트: 초신성
6 단계 하락
----
차트 연도: 2012년
달: 08월
주: 08.13~08.19
순위: 96
제목: LOVE DAY
아티스트: 양요섭
5 단계 하락
----
차트 연도: 2012년
달: 08월
주: 08.13~08.19
순위: 97
제목: Rock Ur Body
아티스트: VIXX (빅스)
0 NEW
----
차트 연도: 2012년
달: 08월
주: 08.13~08.19
순위: 98
제목: HUSH
아티스트: Apink (에이핑크)
10 단계 하락
----
차트 연도: 2012년
달: 08월
주: 08.13~08.19
순위: 99
제목: Friday (T.G.I.Friday`s Brand Song)
아티스트: CNBLUE (씨엔블루)
25 단계 하락
----
차트 연도: 2012년
달: 08월
주: 08.13~08.19
순위: 100
제목: 좋아보여 (Feat. 검정치마)
아티스트: 버벌진트
1 단계 하락
----
08.20~08.26
차트 연도: 2012년
달: 08월
주: 08.20~08.26
순위: 1
제목: 강남스타일
아티스트: 싸이 (PSY)
0 순위 동일
----
차트 연도: 2012년
달: 08월
주: 08.20~08.26
순위: 2
제목: I Need You
아티스트: 허각
0 NEW
----
차트 연도: 2012년
달

주: 08.20~08.26
순위: 88
제목: Home (Feat. 진실 Of Mad Soul Child)
아티스트: 더블케이 (Double K)
15 단계 하락
----
차트 연도: 2012년
달: 08월
주: 08.20~08.26
순위: 89
제목: Good Boy (Feat. 용준형 Of 비스트)
아티스트: 백지영
9 단계 하락
----
차트 연도: 2012년
달: 08월
주: 08.20~08.26
순위: 90
제목: 여름밤
아티스트: 장재인
13 단계 하락
----
차트 연도: 2012년
달: 08월
주: 08.20~08.26
순위: 91
제목: Sexy Lady
아티스트: 장우영
12 단계 하락
----
차트 연도: 2012년
달: 08월
주: 08.20~08.26
순위: 92
제목: 미역국
아티스트: 팬텀
16 단계 하락
----
차트 연도: 2012년
달: 08월
주: 08.20~08.26
순위: 93
제목: 오뚜기 (Rap Feat. 지윤 Of 4Minute)
아티스트: 솔비
0 NEW
----
차트 연도: 2012년
달: 08월
주: 08.20~08.26
순위: 94
제목: 꽃송이가
아티스트: 버스커 버스커
5 단계 하락
----
차트 연도: 2012년
달: 08월
주: 08.20~08.26
순위: 95
제목: Double Up
아티스트: 투엑스
5 단계 하락
----
차트 연도: 2012년
달: 08월
주: 08.20~08.26
순위: 96
제목: I 돈 Care (Feat. 서수민PD)
아티스트: 용감한 녀석들
9 단계 하락
----
차트 연도: 2012년
달: 08월
주: 08.20~08.26
순위: 97
제목: Like Money (Feat. Akon)
아티스트: 원더걸스
13 단계 하락
----
차트 연도: 2012년
달: 08월
주: 08.20~08.26
순위: 98
제목: FANTASTIC BABY
아티스트: BIGBANG
6 단계 하락
----
차트 연도: 2012년
달: 08월
주: 08.20~08.26
순위: 99
제목: 먼 

...

차트 연도: 2012년
달: 11월
주: 11.26~12.02
순위: 55
제목: 걸어 본다
아티스트: B1A4
7 단계 하락
----
차트 연도: 2012년
달: 11월
주: 11.26~12.02
순위: 56
제목: 하얀 겨울
아티스트: 김범수
0 NEW
----
차트 연도: 2012년
달: 11월
주: 11.26~12.02
순위: 57
제목: Humanoids
아티스트: 동방신기 (TVXQ!)
0 NEW
----
차트 연도: 2012년
달: 11월
주: 11.26~12.02
순위: 58
제목: 나에게 초대
아티스트: 이승기
8 단계 상승
----
차트 연도: 2012년
달: 11월
주: 11.26~12.02
순위: 59
제목: 남자가 다 그렇지 뭐
아티스트: 김종국
14 단계 하락
----
차트 연도: 2012년
달: 11월
주: 11.26~12.02
순위: 60
제목: SEXY LOVE
아티스트: 티아라
8 단계 하락
----
차트 연도: 2012년
달: 11월
주: 11.26~12.02
순위: 61
제목: 강남스타일
아티스트: 싸이 (PSY)
8 단계 하락
----
차트 연도: 2012년
달: 11월
주: 11.26~12.02
순위: 62
제목: 잊었니
아티스트: 정준영
25 단계 하락
----
차트 연도: 2012년
달: 11월
주: 11.26~12.02
순위: 63
제목: 걷다
아티스트: 윤건
9 단계 하락
----
차트 연도: 2012년
달: 11월
주: 11.26~12.02
순위: 64
제목: UP (Feat. 박봄 Of 2NE1)
아티스트: 에픽하이 (EPIK HIGH)
13 단계 하락
----
차트 연도: 2012년
달: 11월
주: 11.26~12.02
순위: 65
제목: River
아티스트: 어반자카파
9 단계 하락
----
차트 연도: 2012년
달: 11월
주: 11.26~12.02
순위: 66
제목: 그XX
아티스트: G-DRAGON
9 단계 하락
----
차트 연도: 2012년
달: 11월
주: 11.26~12.02
순위: 67
제

달: 12월
주: 12.03~12.09
순위: 54
제목: 나를 잊지마요
아티스트: 걸스데이
6 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.03~12.09
순위: 55
제목: 24시간후
아티스트: 2BIC(투빅)
18 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.03~12.09
순위: 56
제목: 사랑이 다른 사랑으로 잊혀지네
아티스트: 하림
0 NEW
----
차트 연도: 2012년
달: 12월
주: 12.03~12.09
순위: 57
제목: My Son
아티스트: 유승우
12 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.03~12.09
순위: 58
제목: 사랑아
아티스트: The One (더원)
9 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.03~12.09
순위: 59
제목: Fine Thank You And You?
아티스트: 10cm
6 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.03~12.09
순위: 60
제목: 나빠
아티스트: 별
13 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.03~12.09
순위: 61
제목: 걸어 본다
아티스트: B1A4
6 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.03~12.09
순위: 62
제목: 저녁 하늘
아티스트: 에일리
12 단계 상승
----
차트 연도: 2012년
달: 12월
주: 12.03~12.09
순위: 63
제목: LONELY
아티스트: 스피카
12 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.03~12.09
순위: 64
제목: SEXY LOVE
아티스트: 티아라
4 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.03~12.09
순위: 65
제목: 강남스타일
아티스트: 싸이 (PSY)
4 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.03~12.09
순위: 66
제목: 60초
아티스트: 김성규

달: 12월
주: 12.10~12.16
순위: 54
제목: 있기 없기
아티스트: 달샤벳
5 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.10~12.16
순위: 55
제목: 저녁 하늘
아티스트: 에일리
7 단계 상승
----
차트 연도: 2012년
달: 12월
주: 12.10~12.16
순위: 56
제목: 독 (Feat. E-Sens Of 슈프림팀)
아티스트: 프라이머리
6 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.10~12.16
순위: 57
제목: 너를 위해 (원곡가수 임재범)
아티스트: 이은미
0 NEW
----
차트 연도: 2012년
달: 12월
주: 12.10~12.16
순위: 58
제목: 12월의 이야기
아티스트: 스윗소로우 (SWEET SORROW)
34 단계 상승
----
차트 연도: 2012년
달: 12월
주: 12.10~12.16
순위: 59
제목: 사랑해 봤니
아티스트: J.Lee
13 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.10~12.16
순위: 60
제목: 함께라면 (Feat. 김진표) (JTBC 드라마 `우리 결혼할 수 있을까` 삽입)
아티스트: 클래지콰이
19 단계 상승
----
차트 연도: 2012년
달: 12월
주: 12.10~12.16
순위: 61
제목: 오늘 뭐해?
아티스트: 헬로비너스
0 NEW
----
차트 연도: 2012년
달: 12월
주: 12.10~12.16
순위: 62
제목: 나를 잊지마요
아티스트: 걸스데이
8 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.10~12.16
순위: 63
제목: 난리good!!! (AIR)
아티스트: 개코
0 NEW
----
차트 연도: 2012년
달: 12월
주: 12.10~12.16
순위: 64
제목: 내 맘은 죽어가요 (Feat. Speed)
아티스트: 더 씨야
17 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.10~12.16
순위: 65
제목: My Sweetie
아티

달: 12월
주: 12.17~12.23
순위: 52
제목: 크레용 (Crayon)
아티스트: G-DRAGON
0 순위 동일
----
차트 연도: 2012년
달: 12월
주: 12.17~12.23
순위: 53
제목: 립스틱 (LIPSTICK)
아티스트: 오렌지 캬라멜
2 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.17~12.23
순위: 54
제목: 눈물이 주르륵
아티스트: 손담비
6 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.17~12.23
순위: 55
제목: 아무나 (Feat. 길미)
아티스트: 은지원
0 NEW
----
차트 연도: 2012년
달: 12월
주: 12.17~12.23
순위: 56
제목: 멀어질까봐
아티스트: C-CLOWN
33 단계 상승
----
차트 연도: 2012년
달: 12월
주: 12.17~12.23
순위: 57
제목: 저녁 하늘
아티스트: 에일리
2 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.17~12.23
순위: 58
제목: 눈물이었다
아티스트: 전인혁 (야다)
0 NEW
----
차트 연도: 2012년
달: 12월
주: 12.17~12.23
순위: 59
제목: 있기 없기
아티스트: 달샤벳
5 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.17~12.23
순위: 60
제목: Missing You (Feat. 김윤아 Of 자우림)
아티스트: G-DRAGON
7 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.17~12.23
순위: 61
제목: 오늘 뭐해?
아티스트: 헬로비너스
0 순위 동일
----
차트 연도: 2012년
달: 12월
주: 12.17~12.23
순위: 62
제목: 사랑한다는 말
아티스트: 이승기
12 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.17~12.23
순위: 63
제목: Happy Brand New Year
아티스트: 버벌진트
17 단계 하락
----
차트 연도: 2012년
달: 12

차트 연도: 2012년
달: 12월
주: 12.24~12.30
순위: 50
제목: 눈물이 툭 (Feat. 박경 Of 블락비)
아티스트: 지아
10 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.24~12.30
순위: 51
제목: 서랍정리
아티스트: 빅마마
0 NEW
----
차트 연도: 2012년
달: 12월
주: 12.24~12.30
순위: 52
제목: 춥다 (Feat. 이하이)
아티스트: 에픽하이 (EPIK HIGH)
8 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.24~12.30
순위: 53
제목: 독약 (毒藥) (Feat. 해리 Of 다비치)
아티스트: 더 씨야
14 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.24~12.30
순위: 54
제목: 사랑이 다른 사랑으로 잊혀지네
아티스트: 하림
7 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.24~12.30
순위: 55
제목: 응급실
아티스트: 정준영
9 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.24~12.30
순위: 56
제목: 저녁 하늘
아티스트: 에일리
1 단계 상승
----
차트 연도: 2012년
달: 12월
주: 12.24~12.30
순위: 57
제목: 인연 (원곡가수 이선희)
아티스트: 소향
20 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.24~12.30
순위: 58
제목: 크레용 (Crayon)
아티스트: G-DRAGON
6 단계 하락
----
차트 연도: 2012년
달: 12월
주: 12.24~12.30
순위: 59
제목: 있기 없기
아티스트: 달샤벳
0 순위 동일
----
차트 연도: 2012년
달: 12월
주: 12.24~12.30
순위: 60
제목: Yesterday
아티스트: 진운
0 NEW
----
차트 연도: 2012년
달: 12월
주: 12.24~12.30
순위: 61
제목: From Mark
아티스트: 하동균
35 단계 하락
----
차트 연도: 